# Digital Twin Fairways
This notebook provides an example on how to use the digital twin backend. It runs a simulation based on fairway information system of the Netherlands. You can define sites (with cargo), climate conditions and ships. Ships will transport the goods from A to B.

In [1]:
import sys
sys.path.append(r"D:\01. Projecten\[130878] DTV vaarwegen\digitaltwin-waterway\dtv_backend")

import datetime

import geojson
import simpy
import time
import json
import shapely
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph

# library to load the fairway information network
import dtv_backend.fis
# the simpy processes and objects
import dtv_backend.simple
import dtv_backend.network.network_utilities
import dtv_backend.berthing

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

### Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open('../../dtv_backend/tests/test-berth/config.json') as f:
    config = geojson.load(f)

### Simulation environment
Setup an simulation environment with a time that starts today. You can also choose a date in the past or future. Using the date 0 won't work on windows. 

In [3]:
# Initialize an environment with a real time
now = datetime.datetime.now()
initial_time = now.timestamp()
env = simpy.Environment(initial_time=initial_time)
env.epoch = now

### Network
Here we load the digital twin network. The topological fairway network derived from the Dutch Fairway Information System. The data is processed to be topological connected and usable for transport network analysis.  

In [4]:
## TEMPORARILY LOAD LOCAL FILE ##
def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)

file = r"D:\01. Projecten\[130878] DTV vaarwegen\02. Data\network_digital_twin_v0.3.json"
G = read_json_file(file)

for n in G.nodes:
    G.nodes[n]['geometry'] = shapely.geometry.Point(G.nodes[n]['X'], G.nodes[n]['Y'])
for e in G.edges:
    edge = G.edges[e]
    edge['geometry'] = shapely.wkt.loads(edge['Wkt'])

env.FG = G

### Connect ports to simulation environment
We have a few different entities defined. The port contains cargo and a crane. The crane can be used to load and unload cargo from ships. The site objects (later to be extended with sluices, stopping areas, etc) are geojson features. 

In [5]:
ports = []
for site in config['sites']:
    port = dtv_backend.simple.Port(env, **site['properties'], **site)
    ports.append(port)

### Connect ships to simulation environment
The ships can also contain cargo. The ships can move over the graph. They are instances of the prototype ships from the Rijkswaterstaat ship dataset. You can have multiple copies of the same ship. All ships can work at the same time. 

In [6]:
ships = []
for ship in config['fleet']:
    kwargs = {}
    kwargs.update(ship)
    kwargs.update(ship['properties'])
    # the ship needs to know about the climate
    if 'climate' in config:
        kwargs['climate'] = config['climate']
    kwargs['shift_start_time'] = datetime.time(6, 0)
    kwargs['shift_end_time'] = datetime.time(23, 0)
    
    # add the keywords for berthing
    kwargs.update({'graph': env.FG,
                   'berth_keyword': 'Berth',
                   'edge_distance':'length_m'})
    
    ship = dtv_backend.simple.Ship(env, **kwargs)
    ships.append(ship)

# Example on Berthing

## Select a ship and set a source and destination

In [7]:
# define nodes for source and destination
src = ports[0].node
dst = ports[1].node

# get a ship by means of example
ship = ships[0]

### Set some fixed parameters for berthing

In [8]:
# the maximum deviation from the shortest path to look for berths
max_distance = 1000 # meters

In [9]:
ship.sleep_if_off_duty()
ship.next_on_duty
#ship.next_off_duty

Timestamp('2022-05-14 06:00:00')

## Make the ship travel from src to dst with berthing

In [16]:
ship.move_to(src)
current_node = ship.node

while current_node != dst:
    # if we're not at the dst we must find the next berth (dst included)
    next_berth = ship.find_berth(src_node=current_node,
                                 dst_node=dst,
                                 max_timestamp=ship.next_off_duty,
                                 max_distance=max_distance,
                                 mean_speed=ship.speed,
                                 include_dst=True)
    
    # sail from current to next_berth
    ship.move_to(next_berth)
    print(f"Sailing from {current_node} to {next_berth}.")
    
    # once the berth is reached, wait until the next duty
    ship.sleep_till_next_duty()
    
    # set current to berth to continue
    current_node = next_berth
    

Sailing from 22161408 to Berth229.
Sailing from Berth229 to Berth299.
Sailing from Berth299 to B17568_A.
